In [ ]:
# This produces the dataframe for NFL Team Defesne

In [ ]:
## Required installations
# 

In [1]:
## REQUIRED ACTIONS - Include in a README doc ## 
# modify the season start date in the 'get_current_week' function
# modify the number of weeks if the NFL adds regular season games to the schedule

In [3]:
# import the libraries
import pandas as pd
import numpy as np
import glob
import os
from datetime import datetime
from IPython.display import display
import shutil
import re

In [6]:
# Set Pandas options to display all columns in a single row without wrapping
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)

In [8]:
# Function to calculate the current week of the NFL season
def get_current_week():
    current_date = datetime.now()
    season_start_date = datetime(2024, 9, 4)  # Update for the season start
    current_week = ((current_date - season_start_date).days // 7) + 1
    return current_week

# Define the current NFL year, week, and season type
current_year = datetime.now().year
current_week = get_current_week()
seasontype = 2 if current_week <= 18 else 3  # Regular season or playoffs

In [10]:
# define the years to pull
# nfl.import_weekly_data(years, columns, downcast)
def get_year_range(current_year, current_week, start_year=2017):
    if current_week <= 18:  # Regular season
        return list(range(start_year, current_year + 1))
    else:  # Playoffs
        return list(range(start_year, current_year))

# Use the function
years = get_year_range(current_year, current_week)

In [16]:
def clean_team_boxscore(filepath):
    # Step 1: Read skipping the first header row
    df = pd.read_excel(filepath, skiprows=1)

    # Step 2: Clean columns
    def normalize_col(col):
        return (
            str(col).replace('\n', ' ')
                    .replace('(', '')
                    .replace(')', '')
                    .replace('"', '')
                    .replace('#', '')
                    .replace('$', '')
                    .replace('/', '')
                    .replace('-', ' ')
                    .strip()
                    .lower()
                    .replace('  ', ' ')
        )
    
    cleaned_cols = [normalize_col(col) for col in df.columns]
    df.columns = cleaned_cols

    return df


In [19]:
# Team name to abbreviation map (same as before)
team_abbr_map = {
    'Arizona Cardinals': 'ARI', 'Atlanta Falcons': 'ATL', 'Baltimore Ravens': 'BAL',
    'Buffalo Bills': 'BUF', 'Carolina Panthers': 'CAR', 'Chicago Bears': 'CHI',
    'Cincinnati Bengals': 'CIN', 'Cleveland Browns': 'CLE', 'Dallas Cowboys': 'DAL',
    'Denver Broncos': 'DEN', 'Detroit Lions': 'DET', 'Green Bay Packers': 'GB',
    'Houston Texans': 'HOU', 'Indianapolis Colts': 'IND', 'Jacksonville Jaguars': 'JAX',
    'Kansas City Chiefs': 'KC', 'Las Vegas Raiders': 'LV', 'Los Angeles Chargers': 'LAC',
    'Los Angeles Rams': 'LAR', 'Miami Dolphins': 'MIA', 'Minnesota Vikings': 'MIN',
    'New England Patriots': 'NE', 'New Orleans Saints': 'NO', 'New York Giants': 'NYG',
    'New York Jets': 'NYJ', 'Philadelphia Eagles': 'PHI', 'Pittsburgh Steelers': 'PIT',
    'San Francisco 49ers': 'SF', 'Seattle Seahawks': 'SEA', 'Tampa Bay Buccaneers': 'TB',
    'Tennessee Titans': 'TEN', 'Washington Commanders': 'WAS',
    
    # Legacy names
    'Oakland Raiders': 'LV', 'Washington Redskins': 'WAS', 'Washington Football Team': 'WAS'
}

# Storage
team_yearly_dfs = {}
team_yearly_files = []

for year in years:
    team_file = f"{year}-NFL_Box_Score_Team-Stats.xlsx"
    try:
        print(f"📂 Processing team file: {team_file}")
        df = clean_team_boxscore(team_file)

        # Add 'season' column from year
        df['season'] = year

        # Map team abbreviation
        df['team_abbr'] = df['team'].map(team_abbr_map)

        # Save to dictionary
        team_yearly_dfs[year] = df

        # Optional: save for inspection
        file_name = f"team_stats_{year}_df.xlsx"
        df.to_excel(file_name, index=False)
        team_yearly_files.append(file_name)
        print(f"✅ Saved: {file_name} | Shape: {df.shape}")

    except Exception as e:
        print(f"❌ Error processing {year}: {e}")


📂 Processing team file: 2017-NFL_Box_Score_Team-Stats.xlsx
✅ Saved: team_stats_2017_df.xlsx | Shape: (534, 64)
📂 Processing team file: 2018-NFL_Box_Score_Team-Stats.xlsx
✅ Saved: team_stats_2018_df.xlsx | Shape: (534, 65)
📂 Processing team file: 2019-NFL_Box_Score_Team-Stats.xlsx
✅ Saved: team_stats_2019_df.xlsx | Shape: (534, 64)
📂 Processing team file: 2020-NFL_Box_Score_Team-Stats.xlsx
✅ Saved: team_stats_2020_df.xlsx | Shape: (538, 64)
📂 Processing team file: 2021-NFL_Box_Score_Team-Stats.xlsx
✅ Saved: team_stats_2021_df.xlsx | Shape: (570, 64)
📂 Processing team file: 2022-NFL_Box_Score_Team-Stats.xlsx
✅ Saved: team_stats_2022_df.xlsx | Shape: (568, 65)
📂 Processing team file: 2023-NFL_Box_Score_Team-Stats.xlsx
✅ Saved: team_stats_2023_df.xlsx | Shape: (570, 65)
📂 Processing team file: 2024-NFL_Box_Score_Team-Stats.xlsx
✅ Saved: team_stats_2024_df.xlsx | Shape: (570, 65)


In [25]:
for year, df in team_yearly_dfs.items():
    print(f"\n📅 Validating {year}...")

    # 1. Missing team_abbr
    missing_abbr = df['team_abbr'].isna().sum()
    if missing_abbr == 0:
        print("✅ All team_abbr values present.")
    else:
        print(f"❌ Missing team_abbr: {missing_abbr}")

    # 2. Duplicates per season+week+game_id+team_abbr
    dup_keys = ['season', 'week', 'game id', 'team_abbr']
    duplicates = df.duplicated(subset=dup_keys).sum()
    if duplicates == 0:
        print("✅ No duplicate season/week/game id/team_abbr rows.")
    else:
        print(f"❌ Duplicate rows found: {duplicates}")

    # 3. Unique game_id should have exactly 2 teams
    game_counts = df['game id'].value_counts()
    bad_games = game_counts[game_counts != 2]
    if len(bad_games) == 0:
        print("✅ All game ids have exactly 2 teams.")
    else:
        print(f"❌ {len(bad_games)} game_ids do not have 2 teams.")

    # 4. Optional: Nulls in core fields
    core_nulls = df[['season', 'week', 'team', 'team_abbr']].isna().sum().sum()
    print(f"ℹ️ Nulls in core fields: {core_nulls}")



📅 Validating 2017...
✅ All team_abbr values present.
✅ No duplicate season/week/game id/team_abbr rows.
✅ All game ids have exactly 2 teams.
ℹ️ Nulls in core fields: 0

📅 Validating 2018...
✅ All team_abbr values present.
✅ No duplicate season/week/game id/team_abbr rows.
✅ All game ids have exactly 2 teams.
ℹ️ Nulls in core fields: 0

📅 Validating 2019...
✅ All team_abbr values present.
✅ No duplicate season/week/game id/team_abbr rows.
✅ All game ids have exactly 2 teams.
ℹ️ Nulls in core fields: 0

📅 Validating 2020...
✅ All team_abbr values present.
✅ No duplicate season/week/game id/team_abbr rows.
✅ All game ids have exactly 2 teams.
ℹ️ Nulls in core fields: 0

📅 Validating 2021...
✅ All team_abbr values present.
✅ No duplicate season/week/game id/team_abbr rows.
✅ All game ids have exactly 2 teams.
ℹ️ Nulls in core fields: 0

📅 Validating 2022...
✅ All team_abbr values present.
✅ No duplicate season/week/game id/team_abbr rows.
✅ All game ids have exactly 2 teams.
ℹ️ Nulls in c

In [27]:
team_stats_combined_df = pd.concat(team_yearly_dfs.values(), ignore_index=True)

# Optional: Save
start_year, end_year = min(years), max(years)
combined_filename = f"team_stats_{start_year}_{end_year}_df.xlsx"
team_stats_combined_df.to_excel(combined_filename, index=False)

print(f"✅ Combined shape: {team_stats_combined_df.shape}")
print(f"💾 Saved: {combined_filename}")


✅ Combined shape: (4418, 68)
💾 Saved: team_stats_2017_2024_df.xlsx


In [33]:
def clean_column_dfs(col):
    """
    Cleans and flattens multi-index column names for DFS salary Excel files:
    - Joins tuples if multi-index
    - Removes special characters
    - Normalizes spaces
    - Converts to lowercase for matching
    """
    if isinstance(col, tuple):
        col = ' '.join(str(x) for x in col if x)

    return (
        str(col)
        .replace('\n', ' ')
        .replace('(', '')
        .replace(')', '')
        .replace('"', '')
        .replace('#', '')
        .replace('$', '')
        .replace('/', '')
        .replace('-', ' ')
        .strip()
        .lower()
        .replace('  ', ' ')
        .replace('   ', ' ')
    )


In [35]:
def process_dfs_salary_file(filepath, year):
    # Load raw DFS file
    raw = pd.read_excel(filepath, header=[0, 1])
    raw.columns = [clean_column_dfs(col) for col in raw.columns]

    # Filter to defenses: DK uses 'dst'; FD uses 'dst' or 'd'
    raw_filtered = raw[
        raw['position draftkings'].str.lower().isin(['dst']) |
        raw['position fanduel'].str.lower().isin(['dst', 'd'])
    ].copy()

    # Fuzzy mapping logic
    col_map = {}
    for col in raw_filtered.columns:
        col_lower = col.lower()
        if 'game id' in col_lower:
            col_map['game_id'] = col
        elif 'player id' in col_lower:
            col_map['player_id'] = col
        elif 'week' in col_lower:
            col_map['week'] = col
        elif 'team' in col_lower and 'information' in col_lower:
            col_map['team'] = col
        elif 'salary' in col_lower and 'draftkings' in col_lower:
            col_map['dk_salary'] = col
        elif 'salary' in col_lower and 'fanduel' in col_lower:
            col_map['fd_salary'] = col
        elif 'fantasy points' in col_lower and 'draftkings' in col_lower:
            col_map['dk_points'] = col
        elif 'fantasy points' in col_lower and 'fanduel' in col_lower:
            col_map['fd_points'] = col

    # Required columns
    required = ['game_id', 'player_id', 'week', 'team', 'dk_salary', 'fd_salary', 'dk_points', 'fd_points']
    missing = [k for k in required if k not in col_map]
    if missing:
        raise KeyError(f"Missing expected columns: {missing}")

    # Subset and rename
    df = raw_filtered[[col_map[k] for k in required]].copy()
    df.columns = required

    # Add season column
    df['season'] = int(year)

    # Ensure numeric types
    df['week'] = pd.to_numeric(df['week'], errors='coerce')
    for col in ['dk_salary', 'fd_salary', 'dk_points', 'fd_points']:
        df[col] = pd.to_numeric(df[col], errors='coerce')

    # Use player_id as team_abbr (works for D/ST)
    df['team_abbr'] = df['player_id']

    return df


In [37]:
dfs_salary_dfs = {}

for year in years:
    dfs_file = f"NFL-{year}-DFS-Dataset.xlsx"
    try:
        print(f"\n💵 Processing DFS salary file for {year}")
        
        # Process and store
        df_dfs = process_dfs_salary_file(dfs_file, year)
        dfs_salary_dfs[year] = df_dfs

        # Save to Excel
        output_file = f"team_dfs_{year}_df.xlsx"
        df_dfs.to_excel(output_file, index=False)
        print(f"✅ Saved: {output_file} | Shape: {df_dfs.shape}")

    except Exception as e:
        print(f"❌ Error processing {year}: {e}")



💵 Processing DFS salary file for 2017
✅ Saved: team_dfs_2017_df.xlsx | Shape: (532, 10)

💵 Processing DFS salary file for 2018
✅ Saved: team_dfs_2018_df.xlsx | Shape: (532, 10)

💵 Processing DFS salary file for 2019
✅ Saved: team_dfs_2019_df.xlsx | Shape: (532, 10)

💵 Processing DFS salary file for 2020
✅ Saved: team_dfs_2020_df.xlsx | Shape: (532, 10)

💵 Processing DFS salary file for 2021
✅ Saved: team_dfs_2021_df.xlsx | Shape: (564, 10)

💵 Processing DFS salary file for 2022
✅ Saved: team_dfs_2022_df.xlsx | Shape: (566, 10)

💵 Processing DFS salary file for 2023
✅ Saved: team_dfs_2023_df.xlsx | Shape: (568, 10)

💵 Processing DFS salary file for 2024
✅ Saved: team_dfs_2024_df.xlsx | Shape: (567, 10)


In [39]:
# Step 1: Validate
for year, df in dfs_salary_dfs.items():
    print(f"\n📅 Validating DFS data for {year}...")

    # Check core fields
    core_fields = ['season', 'week', 'game_id', 'team_abbr']
    missing_cols = [col for col in core_fields if col not in df.columns]
    if missing_cols:
        print(f"❌ Missing columns: {missing_cols}")
        continue

    # Team_abbr
    missing_abbr = df['team_abbr'].isna().sum()
    if missing_abbr == 0:
        print("✅ All team_abbr values present.")
    else:
        print(f"❌ Missing team_abbr entries: {missing_abbr}")

    # Duplicate check
    dup_rows = df.duplicated(subset=core_fields).sum()
    if dup_rows == 0:
        print("✅ No duplicate season/week/game_id/team_abbr rows.")
    else:
        print(f"❌ Duplicate key rows: {dup_rows}")

    # Nulls in salary
    nulls = df[['dk_salary', 'fd_salary']].isna().sum().sum()
    print(f"ℹ️ Null salary cells: {nulls}")

# Step 2: Merge
team_dfs_combined_df = pd.concat(dfs_salary_dfs.values(), ignore_index=True)

# Save combined
dfs_combined_filename = f"team_dfs_{min(years)}_{max(years)}_df.xlsx"
team_dfs_combined_df.to_excel(dfs_combined_filename, index=False)

print(f"\n✅ Final combined DFS salary shape: {team_dfs_combined_df.shape}")
print(f"💾 Saved: {dfs_combined_filename}")



📅 Validating DFS data for 2017...
✅ All team_abbr values present.
✅ No duplicate season/week/game_id/team_abbr rows.
ℹ️ Null salary cells: 6

📅 Validating DFS data for 2018...
✅ All team_abbr values present.
✅ No duplicate season/week/game_id/team_abbr rows.
ℹ️ Null salary cells: 2

📅 Validating DFS data for 2019...
✅ All team_abbr values present.
✅ No duplicate season/week/game_id/team_abbr rows.
ℹ️ Null salary cells: 0

📅 Validating DFS data for 2020...
✅ All team_abbr values present.
✅ No duplicate season/week/game_id/team_abbr rows.
ℹ️ Null salary cells: 0

📅 Validating DFS data for 2021...
✅ All team_abbr values present.
✅ No duplicate season/week/game_id/team_abbr rows.
ℹ️ Null salary cells: 113

📅 Validating DFS data for 2022...
✅ All team_abbr values present.
✅ No duplicate season/week/game_id/team_abbr rows.
ℹ️ Null salary cells: 2

📅 Validating DFS data for 2023...
✅ All team_abbr values present.
✅ No duplicate season/week/game_id/team_abbr rows.
ℹ️ Null salary cells: 0

📅 V

In [43]:
# 🧠 Use in-memory dataframes if available
if 'team_stats_combined_df' not in locals():
    team_stats_combined_df = pd.read_excel(f"team_stats_{min(years)}_{max(years)}_df.xlsx")
    print("📂 Loaded team stats from Excel.")
else:
    print("✅ Using in-memory team_stats_combined_df")

if 'team_dfs_combined_df' not in locals():
    team_dfs_combined_df = pd.read_excel(f"team_dfs_{min(years)}_{max(years)}_df.xlsx")
    print("📂 Loaded DFS salary from Excel.")
else:
    print("✅ Using in-memory team_dfs_combined_df")

# 🛠 Normalize key column if needed
if 'game id' in team_stats_combined_df.columns:
    team_stats_combined_df.rename(columns={'game id': 'game_id'}, inplace=True)

# 🔍 Drop stale salary/point columns if they exist
for col in ['dk_salary', 'fd_salary', 'dk_points', 'fd_points']:
    if col in team_stats_combined_df.columns:
        team_stats_combined_df.drop(columns=col, inplace=True)

# 🔗 Merge on 4 keys
team_stats_dfs_merged_df = team_stats_combined_df.merge(
    team_dfs_combined_df,
    how='left',
    on=['season', 'week', 'game_id', 'team_abbr']
)

# ✅ Final validation
print(f"\n✅ Merged shape: {team_stats_dfs_merged_df.shape}")
salary_nulls = team_stats_dfs_merged_df[['dk_salary', 'fd_salary']].isna().sum().sum()
print(f"ℹ️ Total salary nulls: {salary_nulls}")

# 💾 Save to Excel
final_merged_filename = f"team_stats_dfs_{min(years)}_{max(years)}_df.csv"
team_stats_dfs_merged_df.to_csv(final_merged_filename, index=False)
print(f"💾 Saved: {final_merged_filename}")


✅ Using in-memory team_stats_combined_df
✅ Using in-memory team_dfs_combined_df

✅ Merged shape: (4418, 74)
ℹ️ Total salary nulls: 719
💾 Saved: team_stats_dfs_2017_2024_df.csv


In [49]:
opp_rename_map = {
    '1_opp': 'opp_pts_q1',
    '2_opp': 'opp_pts_q2',
    '3_opp': 'opp_pts_q3',
    '4_opp': 'opp_pts_q4',
    'rush_opp': 'opp_rush_att',
    'yds_opp': 'opp_rush_yds',
    'td_opp': 'opp_rush_tds',
    'comp_opp': 'opp_pass_comp',
    'att_opp': 'opp_pass_att',
    'yds.1_opp': 'opp_pass_yds',
    'td.1_opp': 'opp_pass_tds',
    'total yards_opp': 'opp_total_yards',
    'total plays_opp': 'opp_total_plays',
    'time of possession_opp': 'opp_time_of_possession',
    'third downs made_opp': 'opp_third_downs_made',
    'third downs attempted_opp': 'opp_third_downs_attempts',
    'fourth downs made_opp': 'opp_fourth_downs_made',
    'fourth downs attempted_opp': 'opp_fourth_downs_attempts',
    'sacks_opp': 'opp_sacks',
    'interceptions made_opp': 'opp_interceptions'
}


In [52]:
# Self-merge on game context
df_merged = team_stats_dfs_merged_df.merge(
    team_stats_dfs_merged_df,
    how='inner',
    on=['season', 'week', 'game_id'],
    suffixes=('', '_opp')
)

# Filter out self-matches
df_merged = df_merged[df_merged['team_abbr'] != df_merged['team_abbr_opp']]


# Extract and rename
df_opp_features = df_merged[list(opp_rename_map.keys())].rename(columns=opp_rename_map)

# Combine with original
team_stats_dfs_merged_features_df = pd.concat(
    [team_stats_dfs_merged_df.reset_index(drop=True), df_opp_features.reset_index(drop=True)],
    axis=1
)

# *** csv file ***
team_stats_dfs_merged_features_df.to_csv("team_stats_dfs_merged_features_df.csv", index=False)
print(f"✅ Final shape with opponent features: {team_stats_dfs_merged_features_df.shape}")
print("💾 Saved as: team_stats_dfs_merged_features_df.csv")


✅ Final shape with opponent features: (4418, 94)
💾 Saved as: team_stats_dfs_merged_features_df.csv


In [60]:
# Step 1: Lowercase map of current columns
col_map = {col.lower().strip(): col for col in team_stats_dfs_merged_features_df.columns}

# Step 2: Define your intended canonical column names (lowercase)
base_keys = ['season', 'week', 'date', 'game_id', 'team', 'team_abbr', 'venue',
             'dk_salary', 'fd_salary', 'dk_points', 'fd_points']

defense_keys = ['sacks', 'opponent fumbles recovered', 'defensive fumble recovery td',
                'interception return td', 'blocked punt/fg return td', 'safeties',
                'blocked kick/punt', 'interceptions made', 'points_allowed']

odds_keys = ['opening_odds', 'opening_spread', 'opening_total',
             'line_move1', 'line_move2', 'line_move3',
             'closing_odds', 'closing_spread', 'closing_total',
             'opening_ml', 'closing_ml']

# Step 3: Resolve real column names from the map
def resolve_keys(keys):
    return [col_map[k] for k in keys if k in col_map]

base_cols = resolve_keys([k.lower() for k in base_keys])
defensive_cols = resolve_keys([k.lower() for k in defense_keys])
odds_cols = resolve_keys([k.lower() for k in odds_keys])
opp_cols = [col for col in team_stats_dfs_merged_features_df.columns if col.startswith('opp_')]

# Step 4: Reorder columns safely
final_order = base_cols + defensive_cols + opp_cols + odds_cols
remaining_cols = [col for col in team_stats_dfs_merged_features_df.columns if col not in final_order]

team_stats_dfs_merged_features_df = team_stats_dfs_merged_features_df[final_order + remaining_cols]

# Step 5: Drop raw quarters and leftover noise
drop_cols = [col for col in team_stats_dfs_merged_features_df.columns if re.fullmatch(r'[1-4]|OT', col)]
drop_cols += ['unnamed: 62', 'opening moneyline', 'closing moneyline', 'start time (et)']

team_stats_dfs_merged_features_df = team_stats_dfs_merged_features_df.drop(columns=drop_cols, errors='ignore')

# Step 6: Save final
start_year = years[0]
end_year = years[-1]
final_filename = f"team_stats_dfs_{start_year}_{end_year}_df.csv"
team_stats_dfs_merged_features_df.to_csv(final_filename, index=False)

print(f"✅ Cleaned & finalized dataframe shape: {team_stats_dfs_merged_features_df.shape}")
print(f"💾 Saved as: {final_filename}")


✅ Cleaned & finalized dataframe shape: (4418, 87)
💾 Saved as: team_stats_dfs_2017_2024_df.csv


In [ ]:
## tasks
# 1. Restore Scoring Per Quarter Columns
# 2. Drop Extra Columns
# 3. Reorder Columns
# 4. Save Final Team DataFrame
# 5. Create Defensive-Only DataFrame
# 6. Prevous blocks used "to_xlsx" - change to "to_csv"
# 7. change the filename to team_df